# This notebook will give a first baseline estimation for the matching of entities via a random forest algorithm as multi-class classification

In [19]:
import os
import pandas as pd
import plotly.express as px
import progressbar
import json
import numpy as np
import nltk
from nltk.corpus import stopwords
import string
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

In [2]:
def remove_stopwords(token_vector, stopwords_list):
    return token_vector.apply(lambda token_list: [word for word in token_list if word not in stopwords_list])

In [3]:
def remove_punctuation(token_vector):
    return token_vector.apply(lambda token_list: [word for word in token_list if word not in string.punctuation])

In [4]:
#get path informationb
product_path = '../../../../src/data/product'
train_test_input_filtered_path = os.path.join(product_path, 'train_test_split/input_filtered_tables')
train_test_all_filtered_path = os.path.join(train_test_input_filtered_path, 'all')

In [5]:
#fet whole data for baseline
all_filtered_tables_df = pd.read_json(os.path.join(train_test_all_filtered_path,'train_test_all_filtered_tables.json.gz'), compression='gzip', lines=True)

In [6]:
# get information about train and test table split
product_path = '../../../../src/data/product'
train_test_output_path = os.path.join(product_path, 'train_test_split/output_unfiltered_tables')
zip_files_train = [file for file in os.listdir(os.path.join(train_test_output_path, 'large/train')) if file.endswith('.json.gz')]
zip_files_test = [file for file in os.listdir(os.path.join(train_test_output_path, 'large/test')) if file.endswith('.json.gz')]

In [7]:
#filter down the dataframe
df_whole = all_filtered_tables_df[['name','description','cluster_id','table_id']]

In [8]:
# fill emtpy values for description and the concat name and description
df_whole.description.fillna(value=',', inplace=True)
df_whole['concat_information']=df_whole['name'].astype(str)#+df_whole['description'].astype(str)
df_whole.drop(columns=['name','description'],inplace= True)

/home/bizer-tp2021/anaconda3/lib/python3.8/site-packages/pandas/core/series.py:4463: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(
/tmp/ipykernel_380550/3294319364.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_whole['concat_information']=df_whole['name'].astype(str)#+df_whole['description'].astype(str)
/home/bizer-tp2021/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pyda

In [9]:
#clean concated description column to use tf-idf 
df_whole['concat_information'] = df_whole['concat_information'].apply(lambda row: row.lower())
df_whole['tokens'] = df_whole['concat_information'].apply(lambda row: word_tokenize(row))
df_whole['tokens'] = remove_stopwords(df_whole['tokens'],stopwords.words())
df_whole['tokens'] = remove_punctuation (df_whole['tokens'])
df_whole.drop(columns=['concat_information'],inplace=True)

/tmp/ipykernel_380550/256944513.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_whole['concat_information'] = df_whole['concat_information'].apply(lambda row: row.lower())
/tmp/ipykernel_380550/256944513.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_whole['tokens'] = df_whole['concat_information'].apply(lambda row: word_tokenize(row))
/tmp/ipykernel_380550/256944513.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,co

In [10]:
#define vectorizer to match preprocessed tokes
def dummy_fun(doc):
    return doc

tfidf = TfidfVectorizer(
    analyzer='word',
    tokenizer=dummy_fun,
    preprocessor=dummy_fun,
    token_pattern=None)  
tfidf_value = tfidf.fit_transform(df_whole['tokens'])

In [15]:
df_tfidf = pd.DataFrame(tfidf_value.toarray(), columns=tfidf.get_feature_names())
df_prepared = pd.concat([df_whole, df_tfidf], axis=1)

In [22]:
df_train = df_prepared[df_prepared['table_id'].isin(zip_files_train)]
df_target_train = df_train['cluster_id']
df_train.drop(columns=['cluster_id','table_id','tokens'],inplace= True)

/home/bizer-tp2021/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [23]:
df_test = df_prepared[df_prepared['table_id'].isin(zip_files_test)]
df_target_test = df_test['cluster_id']
df_test.drop(columns=['cluster_id','table_id','tokens'],inplace= True)

In [25]:
# Baseline random forest
rf = RandomForestClassifier()
rf.fit(df_train,df_target_train)
prediction = rf.predict(df_test) 
f1 = f1_score(df_target_test,prediction)
accuracy = accuracy_score(target_test,prediction) 
precision = precision_score(target_test,prediction) 
recall = recall_score(target_test,prediction) 
print("The F1-Score on test set: {:.4f}".format(f1))
print("The Precision on test set: {:.4f}".format(precision))
print("The Recall on test set: {:.4f}".format(recall))
print("The Accuracy-Score micro on test set: {:.4f}".format(accuracy))

NameError: name 'target_test' is not defined

In [30]:
f1 = f1_score(df_target_test,prediction,average='micro')
accuracy = accuracy_score(df_target_test,prediction) 
precision = precision_score(df_target_test,prediction,average='micro') 
recall = recall_score(df_target_test,prediction,average='micro') 
print("The F1-Score on test set: {:.4f}".format(f1))
print("The Precision on test set: {:.4f}".format(precision))
print("The Recall on test set: {:.4f}".format(recall))
print("The Accuracy-Score micro on test set: {:.4f}".format(accuracy))

The F1-Score on test set: 0.8157
The Precision on test set: 0.8157
The Recall on test set: 0.8157
The Accuracy-Score micro on test set: 0.8157
